# The code provided below is designed to demonstrate the data generated by the program pipeline

To begin with, the method _extract_required_metrics(report_path)_ is created that pulls all the necessary metrics from the initial reports 
and creates a data frame and a convenient table based on them:

In [ ]:
import pandas as pd
import os

# Function to extract required metrics from a report file
def extract_required_metrics(report_path):
    try:
        # Reading the report file
        df = pd.read_csv(report_path, sep='\t', header=None, names=['Metric', 'Value'])
        
        # Dictionary to store extracted metrics
        metrics = {}
        
        # List of key metrics to extract
        required_metrics = [
            ('Total length', 'Total length'),
            ('GC', 'GC (%)'),
            ('Largest contig', 'Largest Contig'),
            ('N50', 'N50'),
            ('N90', 'N90'),
            ('L50', 'L50'),
            ('L90', 'L90'),
            ('# N\'s per 100 kbp', '# N\'s per 100 kbp')
        ]
        
        # Extracting and saving the values of required metrics
        for metric_pattern, metric_name in required_metrics:
            metric_value = df[df['Metric'].str.contains(metric_pattern, case=False, regex=False)]['Value'].values
            if metric_value.size > 0:
                metrics[metric_name] = metric_value[0]
            else:
                metrics[metric_name] = 'N/A'  # Marking as 'N/A' if the metric is missing
        
        return metrics
    except Exception as e:
        print(f"Error processing {report_path}: {e}")
        return None

# Path to the directory containing report files
reports_dir = './reports'

# List of report files
report_files = [file for file in os.listdir(reports_dir) if file.endswith('.tsv')]

# Extracting and saving data from all report files
all_metrics_data = {}
for report_file in report_files:
    # Forming the full path to the report file
    report_path = os.path.join(reports_dir, report_file)
    # Determining the trimming parameter name from the report file name
    trimming_param = report_file.replace('report_scaffolds_', '').replace('.tsv', '')
    # Extracting metrics
    metrics = extract_required_metrics(report_path)
    if metrics:
        all_metrics_data[trimming_param] = metrics

# Converting the data into a DataFrame
all_metrics_df = pd.DataFrame.from_dict(all_metrics_data, orient='index').reset_index()
all_metrics_df.rename(columns={'index': 'Trimming Parameters'}, inplace=True)

# List of trimming parameters in the desired order
sorting_order = [
    "NoTrimming",
    "QualityTrim_Q25",
    "AdapterTrim_Q25",
    "LengthFilter_75",
    "ComplexityFilter",
    "SlidingWindow_4nt_q25",
    "QualitySlidingHybrid_Q25_4nt_q25", 
    "QualityAdapterHybrid_Q25", 
    "LengthComplexityHybrid_75", 
    "SlidingComplexityHybrid_4nt_q25",
    "AdapterSlidingHybrid_Q25_4nt_q25", 
    "QualityLengthHybrid_Q25_75", 
    "QualityComplexityHybrid_Q25", 
    "AdapterLengthHybrid_Q25_75", 
    "AdapterComplexityHybrid_Q25", 
    "LengthSlidingHybrid_75_4nt_Q25", 
]

# Sorting the DataFrame according to the specified order
sorted_metrics_df = all_metrics_df.set_index('Trimming Parameters').reindex(sorting_order).reset_index()

sorted_metrics_df


## After that, we would like to visualize the same data, but in a more expressive way.

To do this, we first define the base values for the **"NoTrimming"** Trimming Parameters, assuming that this is the control group or standard condition to which all other values will be compared. 

Next, we'll deal with data normalization: we'll initialize an empty _rows_list_ to store the normalized rows. In a loop over each row of the original DataFrame, computes the normalized deviations for each metric compared to the base values of the **"NoTrimming"** Trimming Parameters. If the base value of a metric is 0, the normalized deviation is set to 0 to avoid division by zero. 

Next, a new DataFrame _norm_deviations_ is created from the _rows_list_ of dictionaries, where each dictionary is a row with normalized values.

### Data visualisation:

Using _seaborn.heatmap()_, we create a heatmap to visualize the normalized deviations of genome assembly metrics relative to the **"NoTrimming"** method.

In the heatmap, the rows correspond to the different trimming methods (Trimming Parameters) and the columns correspond to the genome assembly metrics.

The colors of the heatmap cells reflect the magnitude of the normalized deviations, centered at 0, where shades of red indicate positive deviation and shades of blue indicate negative deviation.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Prepare the data
# Make sure the data is numerical.
for column in sorted_metrics_df.columns[1:]:  # Skip the first column with the names of trimming methods
    sorted_metrics_df[column] = pd.to_numeric(sorted_metrics_df[column], errors='coerce')

# Find the base values for "NoTrimming"
base_values = sorted_metrics_df[sorted_metrics_df['Trimming Parameters'] == 'NoTrimming'].iloc[0, 1:]

# Initialization of the list for storing strings
rows_list = []

# Cycle to fill the list of rows
for index, row in sorted_metrics_df.iterrows():
    norm_row = {'Trimming Parameters': row['Trimming Parameters']}
    for metric in sorted_metrics_df.columns[1:]:
        if base_values[metric] != 0:
            norm_row[metric] = (row[metric] - base_values[metric]) / base_values[metric]
        else:
            norm_row[metric] = 0
    rows_list.append(norm_row)

# Creating a DataFrame from a list of strings
norm_deviations = pd.DataFrame(rows_list, columns=sorted_metrics_df.columns)

# Visualization of normalized deviations using a heat map
plt.figure(figsize=(12, 8))
sns.heatmap(norm_deviations.iloc[:, 1:], annot=True, cmap='coolwarm', center=0, yticklabels=norm_deviations['Trimming Parameters'])
plt.title('Normalized Deviation of Genome Assembly Metrics from No Trimming')
plt.xlabel('Assembly Metrics')
plt.ylabel('Trimming Methods')
plt.xticks(rotation=45)  # Rotate signatures for better readability
plt.show()


### Analyzing the Normalized Deviation of Genome Assembly Metrics from No Trimming:

First of all, we observe that Total length and GC (%) values remain almost unchanged after applying different trimming methods. This may indicate that different data trimming methods produce consistent results for these key parameters. This is a good sign indicating the reproducibility and reliability of the genome assembly process. This may also indicate that trimming methods do not significantly affect the overall genome profile, at least in terms of total length and GC content. That is, trimming methods can effectively remove low-quality or undesirable portions of the data without losing significant genomic regions. The consistency of these metrics may also reflect the high quality of the original sequencing data. If the sequencing data were of high quality with minimal low-quality regions, trimming may have minimal impact on the overall length and GC content of the assembly.

The second thing that catches the eye is the red color in the heat map in all data where the **Sliding Window** trimming method was used: **SlidingWindow_4nt_q25**, **QualitySlidingHybrid_Q25_4nt_q25**, **SlidingComplexityHybrid_4nt_q25**, **AdapterSlidingHybrid_Q25_4nt_q25**, **LengthSlidingHybrid_75_4nt_Q25**.

Any combination with this trimming method leads to a rather large decrease in the N50 and N90 metrics relative to the value when trimming is not used. This indicates that this trimming method, in any chosen combination, leads to a decrease in genome assembly continuity and accuracy. Most likely, this trimming method may be too aggressive and remove significant portions of the sequenced data. It is possible that this trimming method results in the removal or alteration of DNA regions that are important for proper genome assembly, which affects the final length of the contigs.

In addition, all of the above combinations increase the L50 and L90 values. This means that more contigs are now required to reach 50% or 90% of the total genome assembly length, which may be the result of increased assembly fragmentation. This may indicate a decrease in assembly continuity and is likely due to the removal of significant data and the probable removal of overlaps required to assemble contigs into larger assemblies. 

**What conclusions can be drawn from this?**

1. If we notice an increase in L50 and L90 along with a decrease in N50 and N90 metrics, it is a clear sign that the trimming method leads to a decrease in assembly continuity and quality.
2. In such a case, it is worth reconsidering the trimming parameters or choosing another method that will have less impact on important parts of the data and maintain or improve the continuity of the assembly. For example, the methods named **LengthFilter_75** (filtering by length of reads), **ComplexityFilter** (the percentage of the base that is different from its next base) and their hybrid **LengthComplexityHybrid_75** performed best in terms of reducing the number of N's per 100 kbp relative to the situation when no filters are used (25% improvement). This translates into improved assembly quality, higher continuity, and potentially improved uncertainty.  
3. It is also important to consider the compromise between reducing errors and improving assembly continuity. In some cases, improving one aspect may lead to degradation of the other. For example, all 14 of the 15 combinations (except **LengthSlidingHybrid_75_4nt_Q25**) resulted in a decrease in the frequency of indeterminate nucleotides in the genome assembly for every 100,000 base pairs, but it also resulted in a decrease in the N50 parameter and an even greater decrease in the N90 parameter (an increase in the L50 and L90 parameters, respectively). Only the trimming method called **LengthFilter_75** again performed better than the others and showed the lowest reduction in the N50 parameter.
4. It would be a rational next step to construct an additional Plot to demonstrate the hierarchy of trimming methods with respect to their ratio of the frequency of occurrence of uncertain nucleotides to the parameter N50. To do this, we will create 16 bar plots that display the ratio of # N's per 100 kbp to N50 for each data trimming method and sort them into a descending hierarchy of these values.

In [ ]:
# Calculate the relationship and add it as a new column to the DataFrame multiplying the result by 10,000
sorted_metrics_df['N_Ratio_x10k'] = (sorted_metrics_df['# N\'s per 100 kbp'] / sorted_metrics_df['N50']) * 10000

# Sort the DataFrame by the new column 'N_Ratio'
sorted_df = sorted_metrics_df.sort_values(by='N_Ratio_x10k', ascending=False)

# Visualisation on one common graph
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Trimming Parameters', y='N_Ratio_x10k', data=sorted_df)

plt.title('N_Ratio (# N\'s per 100 kbp / N50) for Different Trimming Methods (Values x10,000)')
plt.xlabel('Trimming Methods')
plt.ylabel('N_Ratio (x10,000)')

# Change the colour of the 'NoTrimming' signature to red
for label in barplot.get_xticklabels():
    if label.get_text() == 'NoTrimming':
        label.set_color('red')

plt.xticks(rotation=45, ha='right')  # Tilt signatures for better readability

plt.tight_layout()
plt.show()

### Analyzing 16 bar plots "N_Ratio (# N\'s per 100 kbp / N50) for Different Trimming Methods (Values x10,000)":

Above all, if we consider the compromise between reducing the continuity of the assembly (decreasing metric N50) and reducing the number of uncertain nucleotides (denoted as 'N') in the genome assembly for every 100,000 base pairs (indicating improved assembly quality, with fewer gaps and uncertain sites in the genomic sequence), we can observe that all trimming methods have performed better than the situation where no trimming method is used, except for all combinations of the **Sliding Window** method. From this point of view, there is no justification for using these trimming methods. On the other hand, the trimming methods **LengthFilter_75**, **ComplexityFilter** and their hybrid **LengthComplexityHybrid_75** show the best result in this metric. 

**What conclusions can be drawn from this?**

1. In such a situation, it would be useful to examine in more detail the two extremes: the worst trimming method (**Sliding Window**) and the best trimming method with different parameters (**Length Filter**) in terms of this compromise. Then compare them in detail with the situation where no trimming is used (**NoTrimming**).
2. We run the programming pipeline again with the same raw sequencing data, but with a variety of different parameters for the two trimming methods selected above. The following 16 parameters for **Sliding Window** & 6 parameters for **Length Filter** were selected for the new run:
   - SlidingWindow_4nt_q15
   - SlidingWindow_4nt_q20
   - SlidingWindow_4nt_q25
   - SlidingWindow_4nt_q30
   - SlidingWindow_7nt_q15
   - SlidingWindow_7nt_q20
   - SlidingWindow_7nt_q25
   - SlidingWindow_7nt_q30
   - SlidingWindow_10nt_q15
   - SlidingWindow_10nt_q20
   - SlidingWindow_10nt_q25
   - SlidingWindow_10nt_q30
   - SlidingWindow_20nt_q15
   - SlidingWindow_20nt_q20
   - SlidingWindow_20nt_q25
   - SlidingWindow_20nt_q30
   - LengthFilter_50
   - LengthFilter_75
   - LengthFilter_100
   - LengthFilter_150
   - LengthFilter_200
   - LengthFilter_250

In [ ]:
# New path to the directory with reports
second_reports_dir = './reports_2'  

# List of report files in a new directory
second_report_files = [file for file in os.listdir(second_reports_dir) if file.endswith('.tsv')]

# Extracting and saving data from all report files
second_all_metrics_data = {}
for report_file in second_report_files:
    # Forming the full path to the report file
    second_report_path = os.path.join(second_reports_dir, report_file)
    # Determining the trimming parameter name from the second report file name
    second_trimming_param = report_file.replace('output_scaffolds_', '').replace('.tsv', '')
    # Extracting metrics
    second_metrics = extract_required_metrics(second_report_path)
    if second_metrics:
        second_all_metrics_data[second_trimming_param] = second_metrics

# Converting the data into a DataFrame
second_all_metrics_df = pd.DataFrame.from_dict(second_all_metrics_data, orient='index').reset_index()
second_all_metrics_df.rename(columns={'index': 'Trimming Parameters'}, inplace=True)

# List of trimming parameters in the desired order
second_sorting_order = [
    "NoTrimming",
    "SlidingWindow_4nt_q15",
    "SlidingWindow_4nt_q20",
    "SlidingWindow_4nt_q25",
    "SlidingWindow_4nt_q30",
    "SlidingWindow_7nt_q15",
    "SlidingWindow_7nt_q20", 
    "SlidingWindow_7nt_q25", 
    "SlidingWindow_7nt_q30", 
    "SlidingWindow_10nt_q15",
    "SlidingWindow_10nt_q20", 
    "SlidingWindow_10nt_q25", 
    "SlidingWindow_10nt_q30", 
    "SlidingWindow_20nt_q15", 
    "SlidingWindow_20nt_q20", 
    "SlidingWindow_20nt_q25",
    "SlidingWindow_20nt_q30",
    "LengthFilter_50",
    "LengthFilter_75",
    "LengthFilter_100",
    "LengthFilter_150",
    "LengthFilter_200",
    "LengthFilter_250",
]

# Sorting the DataFrame according to the specified order
second_sorted_metrics_df = second_all_metrics_df.set_index('Trimming Parameters').reindex(second_sorting_order).reset_index()

second_sorted_metrics_df

## Let's visualize the data, but in a more expressive way again using a heat map:

In [ ]:
# Prepare the data
# Make sure the data is numerical.
for column in second_sorted_metrics_df.columns[1:]:  # Skip the first column with the names of trimming methods
    second_sorted_metrics_df[column] = pd.to_numeric(second_sorted_metrics_df[column], errors='coerce')

base_values = second_sorted_metrics_df[second_sorted_metrics_df['Trimming Parameters'] == 'NoTrimming'].iloc[0, 1:]

# Initialising a list for storing strings
second_rows_list = []

# Cycle to fill the list of rows
for index, row in second_sorted_metrics_df.iterrows():
    second_norm_row = {'Trimming Parameters': row['Trimming Parameters']}
    for metric in second_sorted_metrics_df.columns[1:]:
        if base_values[metric] != 0:
            second_norm_row[metric] = (row[metric] - base_values[metric]) / base_values[metric]
        else:
            second_norm_row[metric] = 0
    second_rows_list.append(second_norm_row)

# Creating a data frame from a list of strings
second_norm_deviations = pd.DataFrame(second_rows_list, columns=second_sorted_metrics_df.columns)

# Visualisation of normalized deviations using a heat map
plt.figure(figsize=(12, 8))
sns.heatmap(second_norm_deviations.iloc[:, 1:], annot=True, cmap='coolwarm', center=0, yticklabels=second_norm_deviations['Trimming Parameters'])
plt.title('Normalised deviations of genome assembly metrics from the "NoTrimming" method')
plt.xlabel('Assembly Metrics after the second pipeline launch')
plt.ylabel('Trimming Methods after the second pipeline launch')
plt.xticks(rotation=45)  # Rotate signatures for better readability
plt.show()


### Analyzing the Normalized Deviation of Genome Assembly Metrics from No Trimming after the second pipeline launch:

We can observe the same traceable pattern as in the previous heatmap: as the # N\'s per 100 kbp_ parameter decreases, the _N50 (N90) parameters also decrease, while the L50 (L90) parameters increase. However, there are situations in which trimming is so terrible that there is an increase in the parameter # N\'s per 100 kbp while the other parameters remain relatively unchanged (**SlidingWindow_7nt_q15** and **SlidingWindow_7nt_q15**), not to mention such sharp reductions in the quality of genomic assembly as with the parameters **SlidingWindow_10nt_q20** and **LengthFilter_250**.

Let's dive deeper into the parameter study again and construct an additional Plot to demonstrate the hierarchy of trimming methods with respect to their ratio of the frequency of occurrence of uncertain nucleotides to the parameter N50. 

In [ ]:
# Calculate the relationship and add it as a new column to the DataFrame multiplying the result by 10,000
second_sorted_metrics_df['N_Ratio_x10k'] = (second_sorted_metrics_df['# N\'s per 100 kbp'] / second_sorted_metrics_df['N50']) * 10000

# Sort the DataFrame by the new column 'N_Ratio'
second_sorted_df = second_sorted_metrics_df.sort_values(by='N_Ratio_x10k', ascending=False)

# Visualisation on one common graph
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Trimming Parameters', y='N_Ratio_x10k', data=second_sorted_df)

plt.title('N_Ratio (# N\'s per 100 kbp / N50) for Different Trimming Methods (Values x10,000) after the second pipeline launch')
plt.xlabel('Trimming Methods')
plt.ylabel('N_Ratio (x10,000)')

# Change the colour of the 'NoTrimming' signature to red
for label in barplot.get_xticklabels():
    if label.get_text() == 'NoTrimming':
        label.set_color('red')

plt.xticks(rotation=45, ha='right')  # Tilt signatures for better readability

plt.tight_layout()
plt.show()

### Analyzing 23 bar plots "N_Ratio (# N\'s per 100 kbp / N50) for Different Trimming Methods (Values x10,000) after the second pipeline launch":

Now we can observe an interesting picture that broadens our understanding of data trimming: the trimming method that we previously called unjustified (**Sliding Window**) after changing certain parameters and running the program pipeline a second time in some cases still justifies itself, i.e. gives a better assembly quality than if trimming had not been applied (**SlidingWindow_7nt_q25**, **SlidingWindow_10nt_q25**, **SlidingWindow_4nt_q15**). And, most amazingly, it performs in some cases (**SlidingWindow_4nt_q20**, **SlidingWindow_20nt_q20**, **SlidingWindow_4nt_q30**) even better than the trimming method we have previously recognized as the best (**LengthFilter_75**). On the other hand, we may observe that the trimming method, which in the previous step has been recognized as the best (**Length Filter**), may, under certain of its parameters, be unreasonable to use (**LengthFilter_250**). 

**What conclusions can be drawn from this?**

1. First, we have experimentally established that not only the choice of data trimming method is important, but also the parameters of this trimming method.
2. Second, we realized that the created program pipeline, combined with intelligent processing and visualization of its results, gave us a more and more precise understanding of which trimming method and its parameters should be used to improve the quality of genomic assembly. Thus, we have created software that allows us to get new data every time we run it until we find the best solution.
3. The next step would be interesting to see how the quality of genomic assembly within each of these two types of trimming varies as a function of the parameters being changed. To do this, we will visualise our data using Scatter Plots.

Now we want to take the **Length Filter** trimming method and use the 2D scatter plot to demonstrate how the N50 and # N's per 100 kbp (on separate 2D scatter plots) of this trimming method change as a function of its parameter. To visualize it we first need to filter the data to include only those rows that correspond to the different **Length Filter** parameters. Then we can create two 2D scatter plots: one for N50 and one for # of N's per 100 kbp.

In [ ]:
# Data filtering for the "Length Filter" trimming method
length_filter_data = second_sorted_metrics_df[second_sorted_metrics_df['Trimming Parameters'].str.contains('LengthFilter')]

# Extracting Length Filter parameters from 'Trimming Parameters' and sorting by them for line continuity
length_filter_params_and_data = length_filter_data['Trimming Parameters'].str.extract('LengthFilter_(\d+)')[0].astype(int)
length_filter_sorted = length_filter_params_and_data.sort_values().index
length_filter_params = length_filter_params_and_data.loc[length_filter_sorted].astype(int)

# Creating a figure and a set of subplots
fig, axs = plt.subplots(1, 2, figsize=(20, 6))  # 1 row, 2 columns

# Plotting N50 with lines connecting points on the first subplot
axs[0].plot(length_filter_params, length_filter_data.loc[length_filter_sorted, 'N50'], marker='o', linestyle='-', color='blue', label='N50')
axs[0].set_title('Variation of N50 depending on the Length Filter parameter')
axs[0].set_xlabel('Length Filter parameter')
axs[0].set_ylabel('N50 value')
axs[0].grid(True)
axs[0].legend()

# Plotting # N's per 100 kbp with lines connecting points on the second subplot
axs[1].plot(length_filter_params, length_filter_data.loc[length_filter_sorted, "# N's per 100 kbp"], marker='o', linestyle='-', color='red', label="# N's per 100 kbp")
axs[1].set_title("Change of # N's per 100 kbp depending on Length Filter parameter")
axs[1].set_xlabel('Length Filter parameter')
axs[1].set_ylabel("# N's per 100 kbp")
axs[1].grid(True)
axs[1].legend()

plt.show()


**What conclusions can be drawn from this?**

1. Now, the inverse relationship between N50 and # N's per 100 kbp is visible.
2. Thanks to this demonstration, we can see how the value of the **Length Filter** method parameter affects the quality of genome assembly.
3. If we run the program pipeline again, applying the **Length Filter** trimming method with parameters around 75-50 values and below, we will likely be able to find an even better way to further enhance the quality of genome assembly.

Now we want to take the **Sliding Window** trimming method and use the 3D scatter plot to demonstrate how the N_Ratio_x10k of this trimming method changes as a function of its parameters. To visualize it we first need to filter the data to include only those rows that correspond to the different **Sliding Window** parameters. Then we can create the 3D scatter plot.

This code below is based on the fact that the trimming parameters for **Sliding Window** include two parameters (like window size and quality threshold) and that they are formatted in a recognizable pattern (e.g., **SlidingWindow_4nt_q20**):


In [ ]:
import plotly.graph_objects as go

# Filter data for "Sliding Window" trimming method
sliding_window_data = second_sorted_metrics_df[second_sorted_metrics_df['Trimming Parameters'].str.startswith('SlidingWindow')]

# Extract and prepare the Sliding Window parameters for plotting
# Assuming the format is "SlidingWindow_[window size]nt_q[quality]"
window_sizes = sliding_window_data['Trimming Parameters'].str.extract('(\d+)nt')[0].astype(int)
quality_thresholds = sliding_window_data['Trimming Parameters'].str.extract('q(\d+)')[0].astype(int)

# Create a Plotly 3D scatter plot for N50
fig = go.Figure(data=[go.Scatter3d(
    x=window_sizes,
    y=quality_thresholds,
    z=sliding_window_data['N_Ratio_x10k'],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8)
)])

fig.update_layout(title='3D Scatter Plot of N_Ratio_x10k for Sliding Window Trimming',
                  scene=dict(xaxis_title='Window Size',
                             yaxis_title='Quality Threshold',
                             zaxis_title='N_Ratio_x10k'))

fig.show(renderer="browser")

**What conclusions can be drawn from this?**

1. Now, by opening this interactive 3D Scatter Plot (in the browser), we can observe each point in space along with its X, Y, and Z values. The values on the X and Y axes represent the numerical parameters of the **Sliding Window** trimming method (window size and mean quality), while the value on the Z axis corresponds to N_Ratio_x10k. The lower this value is, the better the compromise between # N's per 100 kbp and N50, and consequently, the higher the quality of the genome assembly.
2. We can identify the lowest point on this 3D Scatter Plot. This means that its X and Y values represent the optimal window size and mean quality values for this trimming method in terms of their impact on the subsequent genome assembly.
3. It is quite likely that if we initiate a third run of the software pipeline with window size and mean quality values near this point, we can further enhance the trimming quality.

In [ ]:
from IPython.display import Video

video_path = './video/Sliding Window Data Visualisation.mov'

width = 800  
height = 700 

video = Video(video_path, width=width, height=height)
video

In [ ]:
def extract_correct_metrics(report_file_path):
    try:
        df_correct = pd.read_csv(report_file_path, sep='\t', header=0)
        
        correct_metrics = {
            'Total length': df_correct.loc[0, 'Total length'],
            'GC (%)': df_correct.loc[0, 'GC (%)'],
            'Largest Contig': df_correct.loc[0, 'Largest contig'],
            'N50': df_correct.loc[0, 'N50'],
            'N90': df_correct.loc[0, 'N90'],
            'L50': df_correct.loc[0, 'L50'],
            'L90': df_correct.loc[0, 'L90'],
            "# N's per 100 kbp": df_correct.loc[0, "# N's per 100 kbp"]
        }
        
        return correct_metrics
    except Exception as e:
        print(f"Error processing {report_file_path}: {e}")
        return None


# New path to the directory with reports
third_reports_dir = './reports_3'  

# List of report files in a new directory
third_report_files = [file for file in os.listdir(third_reports_dir) if file.endswith('.tsv')]

# Extracting and saving data from all report files
third_all_metrics_data = {}
for report_file in third_report_files:
    # Forming the full path to the report file
    third_report_path = os.path.join(third_reports_dir, report_file)
    # Determining the trimming parameter name from the third report file name
    third_trimming_param = report_file.replace('output_scaffolds_', '').replace('.tsv', '')
    # Extracting metrics
    third_metrics = extract_correct_metrics(third_report_path)
    if third_metrics:
        third_all_metrics_data[third_trimming_param] = third_metrics


# Converting the data into a DataFrame
third_all_metrics_df = pd.DataFrame.from_dict(third_all_metrics_data, orient='index').reset_index()
third_all_metrics_df.rename(columns={'index': 'Trimming Parameters'}, inplace=True)

# List of trimming parameters in the desired order
third_sorting_order = [
    "SlidingWindow_3nt_q30",
    "SlidingWindow_4nt_q30",
    "SlidingWindow_4nt_q29",
    "SlidingWindow_5nt_q30",
    "LengthFilter_30",
    "LengthFilter_35",
    "LengthFilter_40",
    "LengthFilter_45",
    "LengthFilter_50",
    "LengthFilter_55",
]

# Sorting the DataFrame according to the specified order
third_sorted_metrics_df = third_all_metrics_df.set_index('Trimming Parameters').reindex(third_sorting_order).reset_index()

third_sorted_metrics_df

In [ ]:
# Data filtering for the "Length Filter" trimming method
length_filter_data = third_sorted_metrics_df[third_sorted_metrics_df['Trimming Parameters'].str.contains('LengthFilter')]

# Extracting Length Filter parameters from 'Trimming Parameters' and sorting by them for line continuity
length_filter_params_and_data = length_filter_data['Trimming Parameters'].str.extract('LengthFilter_(\d+)')[0].astype(int)
length_filter_sorted = length_filter_params_and_data.sort_values().index
length_filter_params = length_filter_params_and_data.loc[length_filter_sorted].astype(int)

# Creating a figure and a set of subplots
fig, axs = plt.subplots(1, 2, figsize=(20, 6))  # 1 row, 2 columns

# Plotting N50 with lines connecting points on the first subplot
axs[0].plot(length_filter_params, length_filter_data.loc[length_filter_sorted, 'N50'], marker='o', linestyle='-', color='blue', label='N50')
axs[0].set_title('Variation of N50 depending on the Length Filter parameter')
axs[0].set_xlabel('Length Filter parameter')
axs[0].set_ylabel('N50 value')
axs[0].grid(True)
axs[0].legend()

# Plotting # N's per 100 kbp with lines connecting points on the second subplot
axs[1].plot(length_filter_params, length_filter_data.loc[length_filter_sorted, "# N's per 100 kbp"], marker='o', linestyle='-', color='red', label="# N's per 100 kbp")
axs[1].set_title("Change of # N's per 100 kbp depending on Length Filter parameter")
axs[1].set_xlabel('Length Filter parameter')
axs[1].set_ylabel("# N's per 100 kbp")
axs[1].grid(True)
axs[1].legend()

plt.show()

**What conclusions can be drawn from this?**

1. As we can see in the graph, we found no improvement in changing the value of the trimming method near its best previous value.
2. Our hypothesis did not come true.

In [ ]:
# Calculate the relationship and add it as a new column to the DataFrame multiplying the result by 10,000
third_sorted_metrics_df['N_Ratio_x10k'] = (third_sorted_metrics_df['# N\'s per 100 kbp'] / third_sorted_metrics_df['N50']) * 10000

# Sort the DataFrame by the new column 'N_Ratio'
third_sorted_df = third_sorted_metrics_df.sort_values(by='N_Ratio_x10k', ascending=False)

# Visualisation on one common graph
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Trimming Parameters', y='N_Ratio_x10k', data=third_sorted_df)

plt.title('N_Ratio (# N\'s per 100 kbp / N50) for Different Trimming Methods (Values x10,000) after the third pipeline launch')
plt.xlabel('Trimming Methods')
plt.ylabel('N_Ratio (x10,000)')

# Change the colour of the 'NoTrimming' signature to red
for label in barplot.get_xticklabels():
    if label.get_text() == 'SlidingWindow_4nt_q30':
        label.set_color('red')

plt.xticks(rotation=45, ha='right')  # Tilt signatures for better readability

plt.tight_layout()
plt.show()

**What conclusions can be drawn from this?**

1. Our hypothesis came true and we found an even better way to trim the data with new parameters - **SlidingWindow_3nt_q30**.